# 初始化

In [ ]:
import os
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from LLM_API import GLMService, SenseService, KimiService
from dotenv import load_dotenv

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')

# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

service_type = 'zhipu'

def initialize_service(service_type):
    if service_type in ['zhipu', None]:
        version = 'glm-3-turbo'
        #'glm-4' 'glm-4v' 'glm-3-turbo'
        service = GLMService(version)
    elif service_type in ['kimi']:
        version = '8k'
        #'8k'1M/12￥ '32k'1M/24￥ '128k'1M/60￥
        service = KimiService(version)
    elif service_type in ['sensetime']:
        version = 'SenseChat'
        #SenseChat SenseChat-32K SenseChat-128K SenseChat-Turbo SenseChat-FunctionCall
        service = SenseService(version=version)
    else:
        raise ValueError('未知的服务类型')
    
    return service

service = initialize_service(service_type)

# 论文写作

In [ ]:
import json
import re

#————————————————————————————————————————————————————

#Analysis of format

#————————————————————————————————————————————————————

def split_points(text):
    # 使用正则表达式按照序号加句点的格式进行分割
    points = re.findall(r'\d+\.\s+(.+?)(?=\n|\Z)', text, re.DOTALL)
    
    return points

def parse_outline(outline_str):
    try:
        outline_dict = json.loads(outline_str)
    except json.JSONDecodeError:
        print("无法解析提供的一二级目录框架，请确保其是有效的 JSON 格式。")
        return None
    
    # 检查解析后的字典是否符合预期的结构
    if not isinstance(outline_dict, dict):
        print("一二级目录框架不是有效的字典格式。")
        return None

    # 可以进一步处理解析后的 outline_dict，根据需要进行操作，比如验证键值对是否符合预期等
    # 以下是示例代码，你可以根据具体需求进行修改
    chapters = []
    for chapter, sub_chapters in outline_dict.items():
        if sub_chapters is None:  # 如果子章节为 None，表示该大章节下没有小章节
            sub_chapters = ['']   # 添加一个空白的小标题
        elif not isinstance(sub_chapters, list):
            print(f"章节 '{chapter}' 的子章节不是有效的列表格式。")
            continue
        
        # 追加二级章节的关键点作为三级目录
        sub_chapters_with_keypoints = []
        for sub_chapter in sub_chapters:
            if isinstance(sub_chapter, dict) and "title" in sub_chapter and "key_points" in sub_chapter:
                sub_chapters_with_keypoints.append((sub_chapter["title"], sub_chapter["key_points"]))
            else:
                sub_chapters_with_keypoints.append((sub_chapter, []))  # 若未提供关键点，则为空列表
        
        chapters.append((chapter, sub_chapters_with_keypoints))
    
    return chapters

def split_sentences(chapter_content):
    # 初始化结果列表
    sentences = []

    # 使用换行符来分割文段
    lines = chapter_content.split('\n')

    # 遍历每一行文本
    for line in lines:
        # 如果当前行不为空，则添加到结果列表中
        if line.strip():
            sentences.append(line.strip())

    return sentences

#————————————————————————————————————————————————————

#Process_rounds

#————————————————————————————————————————————————————

def get_first_level_outline(theme, service):
    """
    获取论文的一级目录信息。

    参数：
    - theme：论文主题，字符串类型。
    - service：用于与用户交互的服务对象。

    返回值：
    - 一级目录信息，字典类型。
    """
    input_text = f'''
    主题：“{theme}”。请根据该主题，以规范格式设计一篇论文的一二级目录框架。
    请按以下格式提供一二级目录框架，并确保每个二级章节都包含一个标题：
    {{
        "一级章节1": 
            {{
                 "二级章节1":[],
                 "二级章节2":[],
                 ...
            }},
            ...
        ,
        "一级章节2":             
            {{
                 "二级章节1":[],
                 "二级章节2":[]
            }},
        ...
    }}
    '''


    while True:
        first_level_outline = service.ask_once(input_text)
        # 尝试直接解析JSON
        try:
            first_level_outline_json = json.loads(first_level_outline)
            return first_level_outline_json
        except json.JSONDecodeError:
            # 尝试从包裹符号中提取JSON内容再解析
            json_content_match = re.search(r'```json\s*(.*?)\s*```', first_level_outline, re.DOTALL)
            if json_content_match:
                json_content = json_content_match.group(1)
                try:
                    first_level_outline_json = json.loads(json_content)
                    return first_level_outline_json
                except json.JSONDecodeError:
                    print("返回的一级目录框架不是有效的 JSON 格式，请重新生成:",first_level_outline)
                    continue
            else:
                print("返回的一级目录框架不是有效的 JSON 格式，请重新生成。",first_level_outline)
                continue

def get_second_level_outline(theme, first_level_outline, service):
    """
    获取论文的二级目录信息。

    参数：
    - theme：论文主题，字符串类型。
    - first_level_outline：一级目录信息，字典类型。
    - service：用于与用户交互的服务对象。

    返回值：
    - 论文的二级目录信息，字典类型。
    """
    second_level_outline = {}
    for first_level, second_level_list in first_level_outline.items():
        parsed_second_level_list = []
        for second_level_title in second_level_list:
            while True:
                input_prompt = f'''
                主题：{theme}。对于当前章节"{first_level}"，请提供关于"{second_level_title}"这个二级章节的关键点，每个点是一个词：

                请确保你提供的关键点符合以下要求，并以JSON格式返回：
                {{
                    "title": "{second_level_title}",
                    "key_points": ["关键点1", "关键点2", ...]
                }}
                '''
                key_points = service.ask_once(input_prompt)
                try:
                    key_points_json = json.loads(key_points)
                    # 只在成功解析后添加到列表，避免重复添加
                    parsed_second_level_list.append({second_level_title : key_points_json["key_points"]})
                    break
                except json.JSONDecodeError:
                    json_content_match = re.search(r'```json\s*(.*?)\s*```', key_points, re.DOTALL)
                    if json_content_match:
                        json_content = json_content_match.group(1)
                        try:
                            key_points_json = json.loads(json_content)
                            parsed_second_level_list.append({second_level_title : key_points_json["key_points"]})
                            break
                        except json.JSONDecodeError:
                            print("输入的关键点不是有效的 JSON 格式，请重新输入。")
                            continue
                    else:
                        print("无法从输入中提取有效的 JSON 内容，请重新输入。")
                        continue
        second_level_outline[first_level] = parsed_second_level_list
    return second_level_outline

def extract_previous_content(first_level_name, second_level_name, second_level_outline):
    previous_content_list = []

    # 标志位，表示是否找到了指定的一级章节和二级章节
    found = False

    for first_level, second_level_dict in second_level_outline.items():
        if found:
            break

        for second_level, content in second_level_dict.items():
            # 如果找到了指定的一级章节和二级章节
            if first_level == first_level_name and second_level == second_level_name:
                found = True
                break

            # 将当前一级章节和二级章节前面的所有信息组合成字符串并添加到列表中
            previous_content_list.append(f"{first_level}{second_level}{str(content)}")

    return previous_content_list

def extract_key_points_from_outline(outline):
    """
    从二级目录中提取所有三级关键词。
    
    :param outline: 二级目录的结构，一个包含引言和其他部分的字典。
    :return: 一个包含所有三级关键词的列表。
    """
    key_points_list = []
    # 遍历outline中的所有部分
    for section in outline.get('引言', []):
        # 尝试解析key_points字段中的JSON字符串
        try:
            key_points_data = json.loads(section.get('key_points', '{}'))
            # 如果存在key_points字段，则提取关键词
            key_points = key_points_data.get('key_points', [])
            key_points_list.extend(key_points)
        except json.JSONDecodeError:
            # 如果解析失败，跳过该部分
            continue
    return key_points_list

def query_information_based_on_key_points(outline):
    """
    基于二级目录中的关键词进行信息查询。
    
    :param outline: 二级目录的结构，一个包含引言和其他部分的字典。
    """
    # 首先，从outline中提取所有三级关键词
    key_points = extract_key_points_from_outline(outline)
    
    # 示例：打印所有关键词，实际操作中应替换为查询操作
    for key_point in key_points:
        print(f"进行查询：{key_point}")
        # 假设的查询函数，您需要替换为实际的查询操作
        # results = perform_query(key_point)
        # 处理查询结果
        # process_results(results)

def generate_sub_chapter_points(theme, chapter_title, sub_chapters_list, window_length, service):
    """
    生成小章节要点。

    参数：
    - theme：论文主题，字符串类型。
    - chapter_title：大章节标题，字符串类型。
    - sub_chapters_list：小章节标题列表，列表类型。
    - window_length：对话总结长度，整数类型。
    - service：生成内容所用的服务对象。

    返回值：
    - sub_chapter_points：小章节要点列表，列表类型。
    """
    sub_chapter_points = []
    for sub_chapter in sub_chapters_list:
        # 生成小章节要点的请求文本
        input_text = f'主题：“{theme}”，当前大章节：“{chapter_title}”，当前小章节：“{sub_chapter}”。请你作为本专业的专家，根据章节标题，提供论述此小章节的几个要点。'
        # 生成小章节要点
        points = service.ask_once(input_text)
        sub_chapter_points.append(points)
    return sub_chapter_points

def generate_chapter_content(theme, chapter_title, sub_chapter_title,history, keywords, window_length, service):
    """
    生成章节内容。

    参数：
    - theme：论文主题，字符串类型。
    - outline：全文结构，字典类型。
    - chapter_title：当前章节标题，字符串类型。
    - sub_chapter_title：当前小章节标题，字符串类型。
    - window_length：窗口长度，整数类型。
    - service：用于与用户交互的服务对象。

    返回值：
    - 章节内容，列表类型。
    """
    bool, sections_content_summary = summarize_conversation(window_length, history, service)
    print('前文信息为：',sections_content_summary)
    # 构建输入文本
    input_text = f'主题：“{theme}”，当前章节：“{chapter_title}”，当前小章节：“{sub_chapter_title}”。请你作为本专业的专家，根据小章节标题，以学术论文的风格，围绕以下关键点{str(keywords)},详细阐述此小章节的内容。请确保讨论专注且深入，言之有物。知道上文已经论述了{sections_content_summary}，请你继续扩展论述，保持内容的连贯性和一致性。。'
    
    # 与用户交互，获取章节内容
    chapter_content_text = service.ask_once(input_text)
    print('生成章节为：',chapter_content_text)

    return sections_content_summary, chapter_content_text

def generate_markdown_essay(essay_dict):
    markdown_essay = ""

    for first_level, second_level_dict in essay_dict:
        # 一级标题
        markdown_essay += f"# {first_level}\n\n"
        
        for second_level, content in second_level_dict:
            # 二级标题
            markdown_essay += f"## {second_level}\n\n"
            # 正文内容
            markdown_essay += content + "\n\n"
    
    return markdown_essay

#————————————————————————————————————————————————————

#Main_func

#————————————————————————————————————————————————————

def generate_essay(theme, window_length, service):
    """
    生成论文的主函数，接受主题、对话长度、服务对象作为参数，并返回生成的论文内容和预估成本。

    参数：
    - theme：论文主题，字符串类型。
    - window_length：对话总结长度，整数类型。
    - service：生成内容所用的服务对象。

    返回值：
    - essay_full：生成的论文内容，字符串类型。
    - estimated_cost：生成论文的预估成本，浮点数类型。
    """
    # 记录初始使用的token数
    initial_tokens = service.total_tokens_used
    # 初始化论文内容字典

    # 获取论文一级目录
    first_level_outline = get_first_level_outline(theme, service)
    # 获取论文二级目录
    second_level_outline = get_second_level_outline(theme, first_level_outline, service)

    previous_content_list=[]
    essay_dict = {}

    for current_first_level, second_level_list in second_level_outline.items():
        # 遍历每个一级标题及其对应的二级标题和关键词
        current_level_essay = {}
        for section in second_level_list:
            for current_second_level, keywords in section.items():
                # 更新当前的一级标题和二级标题
                previous_content_list.append(f"{current_first_level}：{current_second_level}：关键词为{str(keywords)}")
                current_summary, current_essay_content = generate_chapter_content(theme, current_first_level, current_second_level, previous_content_list, keywords, window_length, service)
                previous_content_list=[current_summary]    
                current_level_essay[current_second_level] = current_essay_content
        
        # 存储当前一级标题下所有二级标题的内容到 essay_dict 中
        essay_dict[current_first_level] = current_level_essay

    # 计算预估成本
    final_tokens = service.total_tokens_used
    tokens_used = final_tokens - initial_tokens
    estimated_cost = tokens_used*0.000005 
    print(essay_dict)
    return essay_dict, estimated_cost


#————————————————————————————————————————————————————

#Summarizer

#————————————————————————————————————————————————————

def summarize_conversation(window_length, conversation_list, service):
    """
    对当前的对话背景信息进行总结，以减少长度，并保证essential_conversations的内容不被修改。
    """
        # 检查对话列表是否为空
    if not conversation_list:
        return False, ''
    essential_conversations = [conversation_list[-1]]
    essential_length = sum(len(conv) for conv in essential_conversations)

    if essential_length > window_length:
        print("[系统]: 输入过长，请减少必要对话内容的长度。")
        return False, ''

    non_essential_conversations = conversation_list[0:-1]
    if not non_essential_conversations:
        final_conversation=''
        for conversation in essential_conversations:
            final_conversation += conversation
        return True, final_conversation

    summarized_content = non_essential_conversations
    while essential_length + sum(len(conv) for conv in summarized_content) > window_length:
        storage = []
        current_summary = ""
        for conv in summarized_content:
            if len(current_summary) + len(conv) > window_length:
                summary = service.ask_once(f"请总结，保留要点即可，越精炼越好：{current_summary}") if current_summary else ""
                storage.append(summary)
                current_summary = conv
            else:
                current_summary += " " + conv

        if current_summary:
            summary = service.ask_once(f"请总结，保留要点即可，越精炼越好：{current_summary}")
            storage.append(summary)
        summarized_content = storage
        
    new_total_length = essential_length + sum(len(conv) for conv in summarized_content)
    if new_total_length <= window_length:
        # 将 summarized_content 列表转换为字符串
        summarized_content_str = ' '.join(summarized_content)
        # 构造最终的字符串，这里假设 conversation_list 的首尾元素已经是字符串
        summarized_conversations = conversation_list[0] + summarized_content_str + conversation_list[-1]
        return True, summarized_conversations
    else:
        # 如果处理后长度仍超过限制，可能需要进一步的处理逻辑或错误处理
        return False, "无法在给定窗口长度内完成总结"

theme = '论述美国对发展数字教育政策和措施'
essay_dict, estimate_cost=generate_essay(theme, 1200, service)


## 解析环节

In [ ]:

for i in essay_dict:
    for j in essay_dict[i]:
        print(j)
def generate_markdown_essay(essay_dict):
    markdown_essay = ""

    for first_level in essay_dict:
        # 一级标题
        markdown_essay += f"# {first_level}\n\n"
        
        for second_level in essay_dict[first_level]:
            content=essay_dict[first_level][second_level]
            # 二级标题
            markdown_essay += f"## {second_level}\n\n"
            # 正文内容
            markdown_essay += content + "\n\n"
    
    return markdown_essay


generate_markdown_essay(essay_dict)